In [1]:
model_id = "openai/clip-vit-base-patch32"

task_spec = dict(
    images="http://images.cocodataset.org/val2017/000000039769.jpg", 
    candidate_labels=[
        "a cat", 
        "two cats", 
        "a dog", 
        "two dogs", 
        "a dog and a cat",
    ], 
)

In [ ]:
# original inference

from transformers import pipeline

pipe = pipeline(
    task="zero-shot-image-classification", 
    model=model_id,
    device_map="auto",
)

pipe(**task_spec)

In [ ]:
# add these additional lines before transformation/configuration to map Conv2d to our alternative version
from dmx.compressor.modeling.nn import experimental
from dmx.compressor.fx.transformer.utils import dmx_aware_mapping

dmx_aware_mapping["torch.nn.modules.conv.Conv2d"] = experimental.Conv2d

# ---
# transformation and configuration
from dmx.compressor import DmxModel

pipe.model = DmxModel.from_torch(pipe.model)
pipe.model.to_basic_mode()

In [ ]:
pipe(**task_spec)

In [ ]:
pipe.model.dmx_config

In [6]:
# QdQ transform pipe.model._gm and show there is no conv2d in encoder
from dmx.compressor.fx.transform import make_compiler_graph
compiler_graph = make_compiler_graph(pipe.model._gm)

In [ ]:
pipe.model._gm.graph.print_tabular()

In [ ]:
compiler_graph.graph.print_tabular()